# Documentation

In [1]:
from IPython.core.display import HTML
HTML("""<style>
    .MathJax_Display { font-size: 23px; }
</style>""")

$$
\theta = \theta + \alpha (r + \gamma max_{a'} Q(s',a') - Q(s,a)) \nabla_{\theta} Q(s,a)
$$

# Setup

In [2]:
"""
Add parent directorys to current path
"""
import os.path
import sys
p = os.path.abspath('..')
if p not in sys.path:
    sys.path.insert(0,p)
p = os.path.abspath('../..')
if p not in sys.path:
    sys.path.insert(0,p)
    
    
"""
Add tiger-env directory to current path
Still not sure why this is needed.
"""
p = os.path.abspath('../../../custom_envs/gym-tiger')
if p not in sys.path:
    sys.path.insert(0, p)
    

"""
Enable hot-reloading
"""    
from notebook_utils import import_module_by_name, reload_module_by_name

def reload():
    reload_module_by_name('experiments.seq_dqn.model', 'SeqDQN')
    global SeqDQN
    from experiments.seq_dqn.model import SeqDQN
    
    
import gym
import gym_tiger
import matplotlib.pyplot as plt
from experiments.seq_dqn.model import SeqDQN, play_one, main, running_avg, plot_running_avg
from tabulate import tabulate


OBS_GROWL_LEFT = [1, 0, 0, 0]
OBS_GROWL_RIGHT = [0, 1, 0, 0]
OBS_START = [0, 0, 1, 0]
OBS_END = [0, 0, 0, 1]

ACTION_OPEN_LEFT = 0
ACTION_OPEN_RIGHT = 1
ACTION_LISTEN = 2
ACTION_MAP = {
    ACTION_OPEN_LEFT: 'OPEN_LEFT',
    ACTION_OPEN_RIGHT: 'OPEN_RIGHT',
    ACTION_LISTEN: 'LISTEN',
}

# Play 1000 Episodes with observation sequence length 2

In [4]:
"""
Remove old tensorboard log files
"""
!rm -rf ./logs

import tensorflow as tf

env = gym.make('Tiger-v0')
env.__init__(reward_tiger=-100, reward_gold=10, reward_listen=-1,
            obs_accuracy=.85)
start_obs = env.reset()
gamma = 0.8
hidden_layer_sizes = [25, 25]
obs_seq_len = 2
copy_period = 25

session = tf.InteractiveSession()
D = obs_seq_len * env.observation_space.n
K = env.action_space.n
model = SeqDQN('main', session, D, K, hidden_layer_sizes, 2,
               start_obs, gamma, log_summaries=True,
               logs_dir='./logs')
tmodel = SeqDQN('target', session, D, K, hidden_layer_sizes, 2,
                start_obs, gamma)
init = tf.global_variables_initializer()
session.run(init)

N = 100
totalrewards = np.zeros(N)

obs_perms = [
    [OBS_START, OBS_START],
    [OBS_START, OBS_GROWL_LEFT],
    [OBS_START, OBS_GROWL_RIGHT],
    [OBS_GROWL_LEFT, OBS_START],
    [OBS_GROWL_LEFT, OBS_GROWL_LEFT],
    [OBS_GROWL_LEFT, OBS_GROWL_RIGHT],
    [OBS_GROWL_RIGHT, OBS_START],
    [OBS_GROWL_RIGHT, OBS_GROWL_LEFT],
    [OBS_GROWL_RIGHT, OBS_GROWL_RIGHT],
]

for n in range(N):
#     eps = 1.0/np.sqrt(n+1)
    eps = 1/(n+1)**(1/5)
    totalreward = play_one(env, model, tmodel, eps, gamma,
                           copy_period)
    totalrewards[n] = totalreward
    if n % 10 == 0:
        ravg = running_avg(totalrewards, n)
        print('episode:', n,
              'total reward:', totalreward,
              'eps:', eps,
              'avg reward (last 100):', ravg)
        Q = []
        for perm in obs_perms:
            o1, o2 = perm
            model.last_n_obs = perm
            _o1 = env.translate_obs(o1)
            _o2 = env.translate_obs(o2)
            pred = model.predict(o2, update=False)
            best_action_idx = np.argmax(pred[0])
            action_values = pred[0].astype(str)
            action_values[best_action_idx] =  '[' + action_values[best_action_idx] + ']'
            a_openl, a_openr, a_listen = action_values
            Q.append([_o1, _o2, a_openl, a_openr, a_listen])
        print(tabulate(Q, headers=['obs1', 'obs2', 'OPEN LEFT Q Value', 'OPEN RIGHT Q Value', 'LISTEN Q Value']))
        display(sorted(model.train_obs_seq_counts.items(), key=lambda x: x[1], reverse=True))
        display(sorted(model.train_obs_seq_action_counts.items(), key=lambda x: x[1], reverse=True))


print('avg reward for last 100 episodes:', totalrewards[-10:].mean())
print('total steps:', totalrewards.sum())
offset = 0

/Users/jblandin/miniconda3/envs/rlpomdp/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


too few experiences 1
too few experiences 2
too few experiences 3
too few experiences 4
episode: 0 total reward: -3258 eps: 1.0 avg reward (last 100): -3258.0
obs1         obs2         OPEN LEFT Q Value      OPEN RIGHT Q Value    LISTEN Q Value
-----------  -----------  -------------------  --------------------  ----------------
START        START        [-0.027944671]                 -0.0452379        -0.0543968
START        GROWL_LEFT   [-0.027944671]                 -0.0452379        -0.0543968
START        GROWL_RIGHT  [-0.027944671]                 -0.0452379        -0.0543968
GROWL_LEFT   START        [-0.027944671]                 -0.0452379        -0.0543968
GROWL_LEFT   GROWL_LEFT   [-0.027944671]                 -0.0452379        -0.0543968
GROWL_LEFT   GROWL_RIGHT  [-0.027944671]                 -0.0452379        -0.0543968
GROWL_RIGHT  START        [-0.027944671]                 -0.0452379        -0.0543968
GROWL_RIGHT  GROWL_LEFT   [-0.027944671]                 -0.0452379

[('GROWL_RIGHT, GROWL_RIGHT', 35),
 ('GROWL_LEFT, GROWL_LEFT', 21),
 ('GROWL_RIGHT, GROWL_LEFT', 20),
 ('GROWL_LEFT, GROWL_RIGHT', 18),
 ('START, GROWL_LEFT', 3)]

[('GROWL_RIGHT, GROWL_RIGHT => LISTEN', 14),
 ('GROWL_LEFT, GROWL_RIGHT => OPEN_RIGHT', 13),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_RIGHT', 11),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_LEFT', 10),
 ('GROWL_RIGHT, GROWL_LEFT => OPEN_LEFT', 10),
 ('GROWL_LEFT, GROWL_LEFT => LISTEN', 8),
 ('GROWL_LEFT, GROWL_LEFT => OPEN_RIGHT', 8),
 ('GROWL_RIGHT, GROWL_LEFT => OPEN_RIGHT', 7),
 ('GROWL_LEFT, GROWL_LEFT => OPEN_LEFT', 5),
 ('GROWL_LEFT, GROWL_RIGHT => LISTEN', 4),
 ('START, GROWL_LEFT => OPEN_LEFT', 3),
 ('GROWL_RIGHT, GROWL_LEFT => LISTEN', 3),
 ('GROWL_LEFT, GROWL_RIGHT => OPEN_LEFT', 1)]

episode: 10 total reward: -4358 eps: 0.6190439206838455 avg reward (last 100): -3253.0
obs1         obs2           OPEN LEFT Q Value  OPEN RIGHT Q Value      LISTEN Q Value
-----------  -----------  -------------------  --------------------  ----------------
START        START                  -0.387337  [-0.36369005]                -0.558058
START        GROWL_LEFT             -0.387337  [-0.36369005]                -0.558058
START        GROWL_RIGHT            -0.387337  [-0.36369005]                -0.558058
GROWL_LEFT   START                  -0.387337  [-0.36369005]                -0.558058
GROWL_LEFT   GROWL_LEFT             -0.387337  [-0.36369005]                -0.558058
GROWL_LEFT   GROWL_RIGHT            -0.387337  [-0.36369005]                -0.558058
GROWL_RIGHT  START                  -0.387337  [-0.36369005]                -0.558058
GROWL_RIGHT  GROWL_LEFT             -0.387337  [-0.36369005]                -0.558058
GROWL_RIGHT  GROWL_LEFT             -0.387337  [-0.36

...

[('GROWL_RIGHT, GROWL_LEFT => OPEN_LEFT', 122),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_LEFT', 114),
 ('GROWL_LEFT, GROWL_LEFT => OPEN_LEFT', 110),
 ('GROWL_RIGHT, GROWL_RIGHT => LISTEN', 95),
 ('GROWL_LEFT, GROWL_LEFT => LISTEN', 94),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_RIGHT', 93),
 ('GROWL_LEFT, GROWL_LEFT => OPEN_RIGHT', 92),
 ('GROWL_LEFT, GROWL_RIGHT => OPEN_RIGHT', 84),
 ('GROWL_LEFT, GROWL_RIGHT => LISTEN', 77),
 ('GROWL_RIGHT, GROWL_LEFT => OPEN_RIGHT', 72),
 ('GROWL_LEFT, GROWL_RIGHT => OPEN_LEFT', 70),
 ('GROWL_RIGHT, GROWL_LEFT => LISTEN', 55),
 ('START, GROWL_LEFT => OPEN_LEFT', 6),
 ('GROWL_RIGHT, START => LISTEN', 4),
 ('GROWL_RIGHT, START => OPEN_LEFT', 4),
 ('START, GROWL_LEFT => OPEN_RIGHT', 4),
 ('START, GROWL_RIGHT => OPEN_LEFT', 3),
 ('GROWL_RIGHT, START => OPEN_RIGHT', 2),
 ('START, GROWL_LEFT => LISTEN', 2),
 ('START, GROWL_RIGHT => LISTEN', 1),
 ('START, GROWL_RIGHT => OPEN_RIGHT', 1),
 ('GROWL_LEFT, START => LISTEN', 1),
 ('GROWL_LEFT, START => OPEN_RIGHT', 1)]

episode: 20 total reward: -3269 eps: 0.5439464428888785 avg reward (last 100): -3453.904761904762
obs1         obs2           OPEN LEFT Q Value  OPEN RIGHT Q Value      LISTEN Q Value
-----------  -----------  -------------------  --------------------  ----------------
START        START                  -0.817949  [-0.7885885]                 -0.929918
START        GROWL_LEFT             -0.817949  [-0.7885885]                 -0.929918
START        GROWL_RIGHT            -0.817949  [-0.7885885]                 -0.929918
GROWL_LEFT   START                  -0.817949  [-0.7885885]                 -0.929918
GROWL_LEFT   GROWL_LEFT             -0.817949  [-0.7885885]                 -0.929918
GROWL_LEFT   GROWL_RIGHT            -0.817949  [-0.7885885]                 -0.929918
GROWL_RIGHT  START                  -0.817949  [-0.7885885]                 -0.929918
GROWL_RIGHT  GROWL_LEFT             -0.817949  [-0.7885885]                 -0.929918
GROWL_RIGHT  GROWL_LEFT             -0.817

[('GROWL_LEFT, GROWL_LEFT', 557),
 ('GROWL_RIGHT, GROWL_RIGHT', 553),
 ('GROWL_RIGHT, GROWL_LEFT', 503),
 ('GROWL_LEFT, GROWL_RIGHT', 460),
 ('START, GROWL_LEFT', 17),
 ('GROWL_RIGHT, START', 14),
 ('START, GROWL_RIGHT', 7),
 ('GROWL_LEFT, START', 6)]

[('GROWL_RIGHT, GROWL_LEFT => OPEN_LEFT', 233),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_LEFT', 212),
 ('GROWL_LEFT, GROWL_LEFT => OPEN_RIGHT', 211),
 ('GROWL_LEFT, GROWL_LEFT => OPEN_LEFT', 202),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_RIGHT', 192),
 ('GROWL_LEFT, GROWL_RIGHT => OPEN_LEFT', 171),
 ('GROWL_RIGHT, GROWL_LEFT => OPEN_RIGHT', 168),
 ('GROWL_LEFT, GROWL_RIGHT => OPEN_RIGHT', 159),
 ('GROWL_RIGHT, GROWL_RIGHT => LISTEN', 149),
 ('GROWL_LEFT, GROWL_LEFT => LISTEN', 144),
 ('GROWL_LEFT, GROWL_RIGHT => LISTEN', 130),
 ('GROWL_RIGHT, GROWL_LEFT => LISTEN', 102),
 ('START, GROWL_LEFT => OPEN_RIGHT', 7),
 ('START, GROWL_LEFT => OPEN_LEFT', 6),
 ('GROWL_RIGHT, START => OPEN_LEFT', 6),
 ('GROWL_LEFT, START => OPEN_RIGHT', 5),
 ('GROWL_RIGHT, START => LISTEN', 4),
 ('GROWL_RIGHT, START => OPEN_RIGHT', 4),
 ('START, GROWL_LEFT => LISTEN', 4),
 ('START, GROWL_RIGHT => OPEN_LEFT', 3),
 ('START, GROWL_RIGHT => LISTEN', 2),
 ('START, GROWL_RIGHT => OPEN_RIGHT', 2),
 ('GROWL_LEFT, START => LISTEN'

KeyboardInterrupt: 

In [159]:
offset = offset + n
for n in range(N):
    eps = 1/(n+offset+1)**(1/5)
    totalreward = play_one(env, model, tmodel, eps, gamma,
                           copy_period, max_steps=10)
    totalrewards[n] = totalreward
    if n % 10 == 0:
        ravg = running_avg(totalrewards, n)
        print('episode:', n,
              'total reward:', totalreward,
              'eps:', eps,
              'avg reward (last 100):', ravg)
        Q = []
        for perm in obs_perms:
            o1, o2 = perm
            model.last_n_obs = perm
            _o1 = env.translate_obs(o1)
            _o2 = env.translate_obs(o2)
            pred = model.predict(o2, update=False)
            best_action_idx = np.argmax(pred[0])
            action_values = pred[0].astype(str)
            action_values[best_action_idx] =  '[' + action_values[best_action_idx] + ']'
            a_openl, a_openr, a_listen = action_values
            Q.append([_o1, _o2, a_openl, a_openr, a_listen])
        print(tabulate(Q, headers=['obs1', 'obs2', 'OPEN LEFT Q Value', 'OPEN RIGHT Q Value', 'LISTEN Q Value']))
        display(sorted(model.train_obs_seq_counts.items(), key=lambda x: x[1], reverse=True))
        display(sorted(model.train_obs_seq_action_counts.items(), key=lambda x: x[1], reverse=True))

print('avg reward for last 100 episodes:', totalrewards[-10:].mean())
print('total steps:', totalrewards.sum())

episode: 0 total reward: 6 eps: 0.39810717055349726 avg reward (last 100): 6.0
obs1         obs2         OPEN LEFT Q Value    OPEN RIGHT Q Value    LISTEN Q Value
-----------  -----------  -------------------  --------------------  ----------------
START        START        -16.99644            -21.975672            [-1.9809632]
START        GROWL_LEFT   1.2844055            [8.018623]            -1.1460425
START        GROWL_RIGHT  -14.89155            -22.86818             [-2.5276563]
GROWL_LEFT   START        -17.425037           -18.718502            [-1.4682394]
GROWL_LEFT   GROWL_LEFT   [9.185982]           7.3551235             1.254421
GROWL_LEFT   GROWL_RIGHT  -17.197468           -15.456255            [-0.96877563]
GROWL_RIGHT  START        -17.623425           -17.68945             [0.03996879]
GROWL_RIGHT  GROWL_LEFT   1.573828             [3.3781276]           -0.979354
GROWL_RIGHT  GROWL_RIGHT  -17.135592           -17.979849            [-3.8057017]


[('START, START', 73),
 ('START, GROWL_RIGHT', 36),
 ('GROWL_RIGHT, START', 29),
 ('START, GROWL_LEFT', 23),
 ('GROWL_LEFT, START', 22),
 ('GROWL_RIGHT, GROWL_RIGHT', 13),
 ('GROWL_LEFT, GROWL_RIGHT', 3),
 ('GROWL_RIGHT, GROWL_LEFT', 3),
 ('GROWL_LEFT, GROWL_LEFT', 1)]

[('START, START => OPEN_RIGHT', 33),
 ('START, START => LISTEN', 28),
 ('START, GROWL_RIGHT => OPEN_RIGHT', 21),
 ('GROWL_RIGHT, START => LISTEN', 17),
 ('GROWL_LEFT, START => LISTEN', 15),
 ('START, GROWL_LEFT => LISTEN', 14),
 ('START, START => OPEN_LEFT', 12),
 ('START, GROWL_RIGHT => LISTEN', 9),
 ('START, GROWL_LEFT => OPEN_RIGHT', 8),
 ('START, GROWL_RIGHT => OPEN_LEFT', 6),
 ('GROWL_RIGHT, START => OPEN_LEFT', 6),
 ('GROWL_RIGHT, START => OPEN_RIGHT', 6),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_RIGHT', 5),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_LEFT', 5),
 ('GROWL_LEFT, START => OPEN_LEFT', 4),
 ('GROWL_LEFT, GROWL_RIGHT => LISTEN', 3),
 ('GROWL_RIGHT, GROWL_LEFT => LISTEN', 3),
 ('GROWL_LEFT, START => OPEN_RIGHT', 3),
 ('GROWL_RIGHT, GROWL_RIGHT => LISTEN', 3),
 ('START, GROWL_LEFT => OPEN_LEFT', 1),
 ('GROWL_LEFT, GROWL_LEFT => LISTEN', 1)]

episode: 10 total reward: -103 eps: 0.39059030831128655 avg reward (last 100): -53.09090909090909
obs1         obs2           OPEN LEFT Q Value  OPEN RIGHT Q Value    LISTEN Q Value
-----------  -----------  -------------------  --------------------  ----------------
START        START                 -19.4583    -22.923761            [-0.9919095]
START        GROWL_LEFT             -5.60674   [4.952983]            -0.9615222
START        GROWL_RIGHT           -16.3059    -24.018963            [-1.8194517]
GROWL_LEFT   START                 -19.6262    -19.640314            [-0.5733648]
GROWL_LEFT   GROWL_LEFT              0.517381  [8.2105255]           3.052517
GROWL_LEFT   GROWL_RIGHT           -18.7661    -16.741123            [-0.39895517]
GROWL_RIGHT  START                 -19.9842    -18.618473            [0.43755558]
GROWL_RIGHT  GROWL_LEFT             -3.50586   [-0.5504595]          -2.0210142
GROWL_RIGHT  GROWL_RIGHT           -18.4188    -19.05718             [-2.9940696]


[('START, START', 75),
 ('START, GROWL_RIGHT', 42),
 ('GROWL_RIGHT, START', 33),
 ('START, GROWL_LEFT', 30),
 ('GROWL_LEFT, START', 26),
 ('GROWL_RIGHT, GROWL_RIGHT', 17),
 ('GROWL_RIGHT, GROWL_LEFT', 5),
 ('GROWL_LEFT, GROWL_RIGHT', 3),
 ('GROWL_LEFT, GROWL_LEFT', 1)]

...

[('START, START => OPEN_RIGHT', 46),
 ('START, GROWL_LEFT => LISTEN', 44),
 ('START, START => LISTEN', 43),
 ('GROWL_RIGHT, START => LISTEN', 39),
 ('GROWL_LEFT, START => LISTEN', 34),
 ('GROWL_RIGHT, GROWL_RIGHT => LISTEN', 29),
 ('START, GROWL_RIGHT => OPEN_RIGHT', 28),
 ('START, GROWL_RIGHT => LISTEN', 26),
 ('START, START => OPEN_LEFT', 22),
 ('GROWL_LEFT, START => OPEN_LEFT', 22),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_LEFT', 18),
 ('GROWL_RIGHT, START => OPEN_LEFT', 15),
 ('GROWL_RIGHT, GROWL_LEFT => LISTEN', 14),
 ('START, GROWL_RIGHT => OPEN_LEFT', 13),
 ('START, GROWL_LEFT => OPEN_RIGHT', 11),
 ('GROWL_RIGHT, START => OPEN_RIGHT', 11),
 ('GROWL_LEFT, GROWL_RIGHT => LISTEN', 9),
 ('GROWL_LEFT, GROWL_LEFT => LISTEN', 9),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_RIGHT', 8),
 ('GROWL_LEFT, START => OPEN_RIGHT', 8),
 ('START, GROWL_LEFT => OPEN_LEFT', 5),
 ('GROWL_RIGHT, GROWL_LEFT => OPEN_LEFT', 3),
 ('GROWL_LEFT, GROWL_LEFT => OPEN_RIGHT', 3),
 ('GROWL_RIGHT, GROWL_LEFT => OPEN_RIGHT', 2

episode: 90 total reward: -100 eps: 0.3501460890033889 avg reward (last 100): -35.21978021978022
obs1         obs2           OPEN LEFT Q Value  OPEN RIGHT Q Value    LISTEN Q Value
-----------  -----------  -------------------  --------------------  ----------------
START        START                  -32.2813   -29.35173             [-1.6387762]
START        GROWL_LEFT              -5.65907  [2.657101]            -1.1301928
START        GROWL_RIGHT            -28.1767   -29.183222            [-2.0708547]
GROWL_LEFT   START                  -33.0474   -24.344137            [-0.89108145]
GROWL_LEFT   GROWL_LEFT              -7.99058  -7.745365             [-0.99069476]
GROWL_LEFT   GROWL_RIGHT            -32.8479   -23.874157            [-0.85201]
GROWL_RIGHT  START                  -31.1187   -25.557198            [-1.1325796]
GROWL_RIGHT  GROWL_LEFT             -12.165    -14.36685             [-2.0935092]
GROWL_LEFT   GROWL_LEFT              -7.99058  -7.745365             [-0.990694

[('START, START', 120),
 ('START, GROWL_RIGHT', 73),
 ('GROWL_RIGHT, START', 72),
 ('GROWL_LEFT, START', 68),
 ('START, GROWL_LEFT', 65),
 ('GROWL_RIGHT, GROWL_RIGHT', 63),
 ('GROWL_RIGHT, GROWL_LEFT', 23),
 ('GROWL_LEFT, GROWL_LEFT', 16),
 ('GROWL_LEFT, GROWL_RIGHT', 10)]

[('START, START => OPEN_RIGHT', 51),
 ('START, START => LISTEN', 47),
 ('START, GROWL_LEFT => LISTEN', 47),
 ('GROWL_RIGHT, START => LISTEN', 43),
 ('GROWL_LEFT, START => LISTEN', 35),
 ('GROWL_RIGHT, GROWL_RIGHT => LISTEN', 33),
 ('START, GROWL_RIGHT => OPEN_RIGHT', 29),
 ('START, GROWL_RIGHT => LISTEN', 29),
 ('GROWL_LEFT, START => OPEN_LEFT', 25),
 ('START, START => OPEN_LEFT', 22),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_LEFT', 21),
 ('GROWL_RIGHT, GROWL_LEFT => LISTEN', 17),
 ('START, GROWL_RIGHT => OPEN_LEFT', 15),
 ('GROWL_RIGHT, START => OPEN_LEFT', 15),
 ('GROWL_RIGHT, START => OPEN_RIGHT', 14),
 ('GROWL_LEFT, GROWL_LEFT => LISTEN', 13),
 ('START, GROWL_LEFT => OPEN_RIGHT', 12),
 ('GROWL_LEFT, GROWL_RIGHT => LISTEN', 9),
 ('GROWL_RIGHT, GROWL_RIGHT => OPEN_RIGHT', 9),
 ('GROWL_LEFT, START => OPEN_RIGHT', 8),
 ('START, GROWL_LEFT => OPEN_LEFT', 6),
 ('GROWL_RIGHT, GROWL_LEFT => OPEN_LEFT', 3),
 ('GROWL_LEFT, GROWL_LEFT => OPEN_RIGHT', 3),
 ('GROWL_RIGHT, GROWL_LEFT => OPEN_RIGHT', 

avg reward for last 100 episodes: -58.1
total steps: -3686.0
